# Elements of Artificial Intelligence and Data Science  2023/2024

## Practical Work/Assignment No. 1 

### Adversarial Search Methods: Minimax for Playing a Board Game 
### Authors:Francisco Carqueija (202205113), Simão Gomes (202304752), Tomás Martins (202304426)

----------------------------------------------------------------------------------------------------------------------------------

In [3]:
# !pip3 install -r requirements.txt

In [4]:
import pygame
import sys
import numpy as np
import random
import copy
import math

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


----------------------------------------------------------------------------------------------------------------------------------

## Spider Line 4: The Problem

- A board game is characterized by the type of board and pieces, the rules of movement of the pieces (operators) and the finishing conditions of the game with the respective score. 
- In this work, the aim is to implement a game for two players and solve different versions of this game, using the Minimax search algorithm with αβ cuts and its variants.
- Human-human, human-computer and computer-computer game modes should be developed, where the computer should exhibit different skills (levels of difficulty). 
- Computer performance should be compared regarding the different skills (e.g., hard, medium, easy), corresponding to different evaluation functions, different depth levels of Minimax, different successor generation ordering and/or variants of the Minimax algorithm. Monte Carlo Tree Search with different configurations may also be use. 
- Emphasis should be placed on the analysis of the results of the computer players (wins, draws, losses, and other quality
parameters, such as the number of plays to obtain the win/loss) and average time spent to obtain the plays.
- All games, if it is possible, should have different versions with variable board size. 
- The application to be developed must have a proper text or graphical user interface, to show the evolution of the board and interact with the user/player. Different game modes must be included, as explained above, allowing the selection of the game mode, type of each player, and skills of computer players. You should allow different skilled computer players to play against each other. 
- You may also consider providing human players with movement “hints”. 

----------------------------------------------------------------------------------------------------------------------------------

## Spider Line 4: The Game

In [5]:
class Board:
    def __init__(self, row_count, col_count):
        self.row_count = row_count
        self.col_count = col_count
        self.board = np.zeros((row_count, col_count))
        self.turn = 0
        self.game_over = False
        self.winning_pieces = []

    def put_piece(self, player, row, col):
        if self.valid_move(row, col):
            self.board[row][col] = player
            return True
        else:
            print("Invalid move!")
            return False

    def valid_move(self, row, col):
        check_sides = 0
        if self.board[row][col] == 0:
            if row == 0 or row == self.row_count - 1:
                return True
            if col == 0 or col == self.col_count - 1:
                return True
            for i in range(col):
                if self.board[row][i] == 0:
                    check_sides += 1
                    break
            for i in range(col + 1, self.col_count):
                if self.board[row][i] == 0:
                    check_sides += 1
                    break
            for i in range(row):
                if self.board[i][col] == 0:
                    check_sides += 1
                    break
            for i in range(row + 1, self.row_count):
                if self.board[i][col] == 0:
                    check_sides += 1
                    break
        else:
            return False
        if check_sides == 4:
            return False
        return True

    def actions(self):
        self.curr_actions = []
        for i in range(self.row_count):
            for j in range(self.col_count):
                if self.valid_move(i, j):
                    self.curr_actions.append([i, j])
        return self.curr_actions

    def win(self, player):
        for c in range(self.col_count - 3):
            for r in range(self.row_count):
                if (
                    self.board[r][c] == player
                    and self.board[r][c + 1] == player
                    and self.board[r][c + 2] == player
                    and self.board[r][c + 3] == player
                ):
                    self.winning_pieces = [[r, c], [r, c + 1], [r, c + 2], [r, c + 3]]
                    return True

        for c in range(self.col_count):
            for r in range(self.row_count - 3):
                if (
                    self.board[r][c] == player
                    and self.board[r + 1][c] == player
                    and self.board[r + 2][c] == player
                    and self.board[r + 3][c] == player
                ):
                    self.winning_pieces = [[r, c], [r + 1, c], [r + 2, c], [r + 3, c]]
                    return True

        for c in range(self.col_count - 3):
            for r in range(3, self.row_count):
                if (
                    self.board[r][c] == player
                    and self.board[r - 1][c + 1] == player
                    and self.board[r - 2][c + 2] == player
                    and self.board[r - 3][c + 3] == player
                ):
                    self.winning_pieces = [[r, c], [r - 1, c + 1], [r - 2, c + 2], [r - 3, c + 3]]
                    return True

        for c in range(self.col_count - 3):
            for r in range(self.row_count - 3):
                if (
                    self.board[r][c] == player
                    and self.board[r + 1][c + 1] == player
                    and self.board[r + 2][c + 2] == player
                    and self.board[r + 3][c + 3] == player
                ):
                    self.winning_pieces = [[r, c], [r + 1, c + 1], [r + 2, c + 2], [r + 3, c + 3]]
                    return True

        return False

    def is_full(self):
        for c in range(self.col_count):
            for r in range(self.row_count):
                if self.board[r][c] == 0:
                    return False
        return True

----------------------------------------------------------------------------------------------------------------------------------

## Spider Line 4: Game Interface

In [6]:
# Constants
WINDOW_WIDTH, WINDOW_HEIGHT = 700, 700
CELL_SIZE = 70  # Adjusted for visual purposes

# Colors
GRAY = (200, 200, 200)
GRAY2 = (72, 73, 75)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Spider Line 4")


def draw_board(board_obj, hovered_cell):
    screen.fill(GRAY2)

    board_width = board_obj.col_count * CELL_SIZE
    board_height = board_obj.row_count * CELL_SIZE

    # Calculate the left and top offsets to center the board
    left_offset = (WINDOW_WIDTH - board_width) // 2
    top_offset = (WINDOW_HEIGHT - board_height) // 2

    for row in range(board_obj.row_count):
        for col in range(board_obj.col_count):
            circle_x = left_offset + col * CELL_SIZE + CELL_SIZE // 2
            circle_y = top_offset + row * CELL_SIZE + CELL_SIZE // 2
            radius = CELL_SIZE // 2 - 2

            is_winning_piece = [row, col] in board_obj.winning_pieces

            pygame.draw.circle(screen, BLACK, (circle_x, circle_y), radius)

            if board_obj.board[row][col] == 1:
                if is_winning_piece:
                    pygame.draw.circle(screen, RED, (circle_x, circle_y), radius)
                    pygame.draw.circle(screen, GREEN, (circle_x, circle_y), radius, 3)
                else:
                    pygame.draw.circle(screen, RED, (circle_x, circle_y), radius)
            elif board_obj.board[row][col] == 2:
                if is_winning_piece:
                    pygame.draw.circle(screen, BLUE, (circle_x, circle_y), radius)
                    pygame.draw.circle(screen, GREEN, (circle_x, circle_y), radius, 3)
                else:
                    pygame.draw.circle(screen, BLUE, (circle_x, circle_y), radius)

            if hovered_cell is not None and [row, col] == hovered_cell:
                if board_obj.valid_move(row, col):
                    if board_obj.turn == 0:
                        pygame.draw.circle(screen, RED, (circle_x, circle_y), radius, 3)
                    else:
                        pygame.draw.circle(screen, BLUE, (circle_x, circle_y), radius, 3)

    pygame.display.flip()


def display_menu():
    font = pygame.font.Font(None, 36)
    title_font = pygame.font.SysFont("consolas", 60)

    title = title_font.render("SpiderLine4", True, BLACK)
    pvp_text = font.render("Player vs Player", True, BLACK)
    pvc_text = font.render("Player vs CPU", True, BLACK)
    cvc_text = font.render("CPU vs CPU", True, BLACK)

    title_rect = title.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 4))
    pvp_rect = pvp_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2))
    pvc_rect = pvc_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 60))
    cvc_rect = cvc_text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 120))

    screen.fill(GRAY)
    screen.blit(title, title_rect)
    pygame.draw.rect(screen, BLACK, pvp_rect.inflate(20, 10), 2)
    screen.blit(pvp_text, pvp_rect)
    pygame.draw.rect(screen, BLACK, pvc_rect.inflate(20, 10), 2)
    screen.blit(pvc_text, pvc_rect)
    pygame.draw.rect(screen, BLACK, cvc_rect.inflate(20, 10), 2)
    screen.blit(cvc_text, cvc_rect)

    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                if pvp_rect.collidepoint(mouse_x, mouse_y):
                    return "pvp"
                elif pvc_rect.collidepoint(mouse_x, mouse_y):
                    return "pvc"
                elif cvc_rect.collidepoint(mouse_x, mouse_y):
                    return "cvc"


def display_board_size_menu():
    font = pygame.font.Font(None, 36)
    title = font.render("Choose Board Size", True, BLACK)
    sizes = [10, 9, 8, 7, 6]
    size_texts = [font.render(f"{size}x{size}", True, BLACK) for size in sizes]

    title_rect = title.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 4))
    size_rects = [text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 60 * i)) for i, text in enumerate(size_texts)]

    screen.fill(GRAY)
    screen.blit(title, title_rect)
    for text, rect in zip(size_texts, size_rects):
        pygame.draw.rect(screen, BLACK, rect.inflate(30, 15), 2)
        screen.blit(text, rect)

    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                for size, rect in zip(sizes, size_rects):
                    if rect.collidepoint(mouse_x, mouse_y):
                        return size

def display_difficulty_menu(message):
    font = pygame.font.Font(None, 36)
    title = font.render(message, True, BLACK)
    difficulties = ["Easy (MCTS)", "Hard (Minimax)", "Hardcore (Negamax)"]
    difficulty_texts = [font.render(difficulty, True, BLACK) for difficulty in difficulties]

    title_rect = title.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 4))
    difficulty_rects = [text.get_rect(center=(WINDOW_WIDTH // 2, WINDOW_HEIGHT // 2 + 60 * i)) for i, text in enumerate(difficulty_texts)]

    screen.fill(GRAY)
    screen.blit(title, title_rect)
    for text, rect in zip(difficulty_texts, difficulty_rects):
        pygame.draw.rect(screen, BLACK, rect.inflate(30, 15), 2)
        screen.blit(text, rect)

    pygame.display.flip()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                for difficulty, rect in zip(difficulties, difficulty_rects):
                    if rect.collidepoint(mouse_x, mouse_y):
                        return difficulty.lower().replace(" ", "_")



----------------------------------------------------------------------------------------------------------------------------------

## Spider Line 4: Heuristic

In [7]:
class Heuristic():

    def score_change(self,c1,c2):
        if c2==0:
            if c1==4: return -80000
            elif c1==3: return -500
            elif c1==2: return -100
            elif c1==1: return -10
        if c1==0:
            if c2==4: return 1000000
            elif c2==3: return 500
            elif c2==2: return 100
            elif c2==1: return 10 
        return 0

    def hot_pos(self,board,player_1,player_2,ROW_COUNT,COL_COUNT):
        if ROW_COUNT==6:
            board_score_matrix = np.array([
                    [3, 4, 6, 6, 4, 3],
                    [4, 5, 7, 7, 5, 4],
                    [6, 7, 8, 8, 7, 6],
                    [6, 7, 8, 8, 7, 6],
                    [4, 5, 7, 7, 5, 4],
                    [3, 4, 6, 6, 4, 3]
                ])

        elif ROW_COUNT==7:
            board_score_matrix = np.array([
                    [3, 4, 5, 7, 5, 4, 3],
                    [4, 6, 8, 9, 8, 6, 4],
                    [5, 8, 10, 11, 10, 8, 5],
                    [7, 9, 11, 12, 11, 9, 7],
                    [5, 8, 10, 11, 10, 8, 5],
                    [4, 6, 8, 9, 8, 6, 4],
                    [3, 4, 5, 7, 5, 4, 3]           
                ])

        elif ROW_COUNT==8:
            board_score_matrix = np.array([
                    [3, 4, 5, 7, 7, 5, 4, 3],
                    [4, 6, 8, 9, 9, 8, 6, 4],
                    [5, 8, 10, 11, 11, 10, 8, 5],
                    [7, 9, 11, 12, 12, 11, 9, 7],
                    [7, 9, 11, 12, 12, 11, 9, 7],
                    [5, 8, 10, 11, 11, 10, 8, 5],
                    [4, 6, 8, 9, 9, 8, 6, 4],
                    [3, 4, 5, 7, 7, 5, 4, 3]          
                ])

        elif ROW_COUNT==9:
            board_score_matrix = np.array([
                    [3, 4, 5, 7, 9, 7, 5, 4, 3],
                    [4, 6, 8, 10, 12, 10, 8, 6, 4],
                    [5, 8, 11, 13, 14, 13, 11, 8, 5],
                    [7, 10, 13, 15, 16, 15, 13, 10, 7],
                    [9, 12, 14, 16, 17, 16, 14, 12, 9],
                    [7, 10, 13, 15, 16, 15, 13, 10, 7],
                    [5, 8, 11, 13, 14, 13, 11, 8, 5],
                    [4, 6, 8, 10, 12, 10, 8, 6, 4],
                    [3, 4, 5, 7, 9, 7, 5, 4, 3]          
                ])

        elif ROW_COUNT==10:
            board_score_matrix = np.array([
                    [3, 4, 5, 7, 9, 9, 7, 5, 4, 3],
                    [4, 6, 8, 10, 12, 12, 10, 8, 6, 4],
                    [5, 8, 11, 13, 14, 14, 13, 11, 8, 5],
                    [7, 10, 13, 15, 16, 16, 15, 13, 10, 7],
                    [9, 12, 14, 16, 17, 17, 16, 14, 12, 9],
                    [9, 12, 14, 16, 17, 17, 16, 14, 12, 9],
                    [7, 10, 13, 15, 16, 16, 15, 13, 10, 7],
                    [5, 8, 11, 13, 14, 14, 13, 11, 8, 5],
                    [4, 6, 8, 10, 12, 12, 10, 8, 6, 4],
                    [3, 4, 5, 7, 9, 9, 7, 5, 4, 3]          
                ])
        
        score = 0
        for r in range (ROW_COUNT):
            for c in range (COL_COUNT):
                if board.board[r][c]==player_1: score-=board_score_matrix[r][c]
                elif board.board[r][c]==player_2: score+=board_score_matrix[r][c]

        return score


    def evaluate_function(self,board,player1,player2,ROW_COUNT,COL_COUNT): #função que permite avaliar a qualidade de uma jogada, avaliando o estado do tabuleiro após essa jogada ser efetuada
        
        score = 0

        # Verificar horizontal
        for c in range(COL_COUNT - 3):
            for r in range(ROW_COUNT):
                c1=0
                c2=0
                if board.board[r][c] == player1: c1+=1 #número de peças de cada jogador nesses 4 espaços do tabuleiro
                elif board.board[r][c] == player2: c2+=1
                if board.board[r][c + 1] == player1: c1+=1
                elif board.board[r][c + 1] == player2: c2+=1
                if board.board[r][c + 2] == player1: c1+=1
                elif board.board[r][c + 2] == player2: c2+=1
                if board.board[r][c + 3] == player1: c1+=1
                elif board.board[r][c + 3] == player2: c2+=1
                score+=self.score_change(c1,c2)
                
        
        # Verificar vertical
        for c in range(COL_COUNT):
            for r in range(ROW_COUNT - 3):
                c1=0
                c2=0
                if board.board[r][c] == player1: c1+=1 #número de peças de cada jogador nesses 4 espaços do tabuleiro
                elif board.board[r][c] == player2: c2+=1
                if board.board[r + 1][c] == player1: c1+=1
                elif board.board[r + 1][c] == player2: c2+=1
                if board.board[r + 2][c] == player1: c1+=1
                elif board.board[r + 2][c] == player2: c2+=1
                if board.board[r + 3][c] == player1: c1+=1
                elif board.board[r + 3][c] == player2: c2+=1
                score+=self.score_change(c1,c2)


        # Verificar diagonal com declive positivo
        for c in range(COL_COUNT - 3):
            for r in range(3, ROW_COUNT):
                c1=0
                c2=0
                if board.board[r][c] == player1: c1+=1 #número de peças de cada jogador nesses 4 espaços do tabuleiro
                elif board.board[r][c] == player2: c2+=1
                if board.board[r - 1][c + 1] == player1: c1+=1
                elif board.board[r - 1][c + 1] == player2: c2+=1
                if board.board[r - 2][c + 2] == player1: c1+=1
                elif board.board[r - 2][c + 2] == player2: c2+=1
                if board.board[r - 3][c + 3] == player1: c1+=1
                elif board.board[r - 3][c + 3] == player2: c2+=1
                score+=self.score_change(c1,c2)


        # Verificar diagonal com declive negativo
        for c in range(COL_COUNT - 3):
            for r in range(ROW_COUNT - 3):
                c1=0
                c2=0
                if board.board[r][c] == player1: c1+=1 #número de peças de cada jogador nesses 4 espaços do tabuleiro
                elif board.board[r][c] == player2: c2+=1
                if board.board[r + 1][c + 1] == player1: c1+=1
                elif board.board[r + 1][c + 1] == player2: c2+=1
                if board.board[r + 2][c + 2] == player1: c1+=1
                elif board.board[r + 2][c + 2] == player2: c2+=1
                if board.board[r + 3][c + 3] == player1: c1+=1
                elif board.board[r + 3][c + 3] == player2: c2+=1
                score+=self.score_change(c1,c2)
        
        return score     
    
    def final_heuristic(self,board,player_1,player_2,ROW_COUNT,COL_COUNT):
        
            eval_score = self.hot_pos(board,player_1,player_2,ROW_COUNT,COL_COUNT)
            board_score = self.evaluate_function(board,player_1,player_2,ROW_COUNT,COL_COUNT)
            total_score = eval_score + board_score                           
            return total_score

In [8]:
heuristic = Heuristic()

----------------------------------------------------------------------------------------------------------------------------------

## Connect Four: Minimax Algorithm

In [9]:
def minimax(board, depth, current_player, ROW_COUNT, COL_COUNT, player_1, player_2, alpha=float('-inf'), beta=float('inf'), history_table=None):
    if depth == 0 or board.win(player_1) or board.win(player_2) or board.is_full():
        return heuristic.final_heuristic(board, player_1, player_2, ROW_COUNT, COL_COUNT), -1, -1

    maximizing_player = current_player == player_2
    if maximizing_player:
        max_eval = float('-inf')
        best_row = None
        best_col = None

        # Order moves based on history table
        actions = board.actions()
        if history_table:
            actions.sort(key=lambda move: history_table.get(move, 0), reverse=True)

        for r, c in actions:
            new_board = copy.deepcopy(board)
            new_board.put_piece(player_2, r, c)
            eval, _, _ = minimax(new_board, depth - 1, False, ROW_COUNT, COL_COUNT, player_1, player_2, alpha, beta, history_table)
            if eval > max_eval:
                best_row = r
                best_col = c
                max_eval = eval
            alpha = max(alpha, eval)
            if beta <= alpha:
                break

        # Update history table
        if history_table and max_eval >= beta:
            history_table[(best_row, best_col)] = depth

        return max_eval, best_row, best_col

    else:
        min_eval = float('inf')
        best_row = None
        best_col = None

        # Order moves based on history table
        actions = board.actions()
        if history_table:
            actions.sort(key=lambda move: history_table.get(move, 0), reverse=True)

        for r, c in actions:
            new_board = copy.deepcopy(board)
            new_board.put_piece(player_1, r, c)
            eval, _, _ = minimax(new_board, depth - 1, True, ROW_COUNT, COL_COUNT, player_1, player_2, alpha, beta, history_table)
            if eval < min_eval:
                best_row = r
                best_col = c
                min_eval = eval
            beta = min(beta, eval)
            if beta <= alpha:
                break

        # Update history table
        if history_table and min_eval <= alpha:
            history_table[(best_row, best_col)] = depth

        return min_eval, best_row, best_col

----------------------------------------------------------------------------------------------------------------------------------

## Connect Four: Negamax Algorithm

In [10]:
def negamax(board, depth, current_player, ROW_COUNT, COL_COUNT, player_1, player_2, alpha=float('-inf'), beta=float('inf')):
    if depth == 0 or board.win(player_1) or board.win(player_2) or board.is_full():
        heuristic = Heuristic()
        return heuristic.final_heuristic(board, player_1, player_2, ROW_COUNT, COL_COUNT) * (-1 if current_player == player_1 else 1), -1, -1
    
    max_eval = float('-inf')
    best_col = -1
    best_row = -1

    for r in range(ROW_COUNT):
        for c in range(COL_COUNT):
            if [r, c] in board.actions():
                new_board = copy.deepcopy(board)
                new_board.put_piece(current_player, r, c)
                eval, _, _ = negamax(new_board, depth - 1, player_2 if current_player == player_1 else player_1, ROW_COUNT, COL_COUNT, player_1, player_2, -beta, -alpha)
                eval = -eval  # Invert the evaluation for the current player

                if eval > max_eval:
                    max_eval = eval
                    best_row = r
                    best_col = c
                
                alpha = max(alpha, eval)
                
                if beta <= alpha:
                    break  # Beta cut-off

    return max_eval, best_row, best_col

----------------------------------------------------------------------------------------------------------------------------------

## Spider Line 4: MCTS Algorithm

In [11]:

class Node:
    def __init__(self, board, player, row, col, row_move = None, col_move = None , parent=None):
        self.row = row
        self.col = col
        self.board = board  #Instancia da classe board 
        self.parent = parent
        self.children = []
        self.row_move = row_move
        self.col_move = col_move
        self.wins = 0
        self.visits = 0
        self.player = player

        

    def is_leaf(self):
        if (len(self.children) == 0) or self.board.win(1) or self.board.win(2) or self.board.is_full():
            return True
        else:
            return False
    def is_terminal(self):
        if self.board.win(1) or self.board.win(2) or self.board.is_full():
            return True
        else:
            return False
    def has_unexplored_moves(self):
        unexplored_moves = self.board.actions()
        print(unexplored_moves)
        if unexplored_moves: 
            return True
        else:
            return False
    
    def expand(self):
        # Identifica as jogadas possíveis que ainda não foram exploradas
        unexplored_moves = self.board.actions()
        
        if unexplored_moves:
            # Escolhe uma jogada não explorada aleatoriamente para a expansão
            move = random.choice(unexplored_moves)
            row_move = move[0]
            col_move = move[1]
            
            # Cria uma cópia do estado do tabuleiro e aplica a jogada escolhida
            new_board = copy.deepcopy(self.board)
            new_board.put_piece(self.player, row_move, col_move)
            
            # Cria um novo nó filho com o estado resultante e adiciona à lista de filhos
            new_node = Node(board=new_board, player= self.player, row = self.row, col = self.col, row_move=row_move, col_move=col_move, parent=self)
            self.children.append(new_node)
            
            # Atualiza as jogadas não exploradas no nó pai
            self.board = new_board
            
            # Retorna o novo nó para que seja utilizado na simulação
            return new_node
        
        # Retorna None se não houver mais movimentos não explorados
        return self


    def select_child(self, c):
        best_score = -float("inf")
        best_children = []
        unvisited_children = []

        for child in self.children:
            if child.visits == 0:
                unvisited_children.append(child)
            else:
                exploration_term = math.sqrt((math.log(self.visits + 1) * 2) / child.visits)
                score = child.wins / child.visits + c * exploration_term
                if score == best_score:
                    best_children.append(child)
                elif score > best_score:
                    best_score = score
                    best_children = [child]

        if len(unvisited_children) > 0:
            return random.choice(unvisited_children)
        else:
            return random.choice(best_children)

    
    def backpropagate(self, result, player):
        self.visits += 1
        if self.player == player:
            self.wins += result
        else:
            self.wins += 1 - result
        if self.parent is not None:
            self.parent.backpropagate(result, player)

In [12]:
def monte_carlo_tree_search(board, player,c, simulations, row, col):
    # Passo 1: Inicialize a árvore
    root = Node(board, player, row, col)

    count = 0
    for _ in range(simulations):
        node = root
        
        while not node.has_unexplored_moves():
            node = node.select_child(c)
        
        if not node.is_terminal():
            node = node.expand()
            count+= 1

        if not node.is_terminal(): 
            result = simulate_random_playout(node.board, player)
            node.backpropagate(result, player)

        else: 
            if(node.board.win(3-player)):
                node.backpropagate(0,3-player)
            elif (node.board.win(player)):
                node.backpropagate(1, player)
            elif(node.board.is_full()):
                node.backpropagate(0.5, player)

        

    best_ratio = -float("inf")
    best_row = None
    best_col = None

    for child in root.children:
        if child.visits > 0:
            ratio = child.wins / child.visits
            print(f"{ratio} Linha:{child.row_move} Coluna: {child.col_move} Numero de Vitorias: {child.wins} Numero de Visitas: {child.visits}")
        else:
            ratio = 0
        if ratio > best_ratio:
            best_ratio = ratio
            best_row = child.row_move
            best_col = child.col_move
    print(f"Numero de Expansoes:  {count}")
    #Retorna o movimento do melhor filho
    return best_row, best_col
        
    
def simulate_random_playout(game_state, player):
    simulated_game = copy.deepcopy(game_state)
    current_player = player
    while not simulated_game.is_full() and not simulated_game.win(current_player):
        possible_moves = simulated_game.actions()
        move = random.choice(possible_moves)
        row_move = move[0]
        col_move = move[1]
        simulated_game.put_piece(current_player, row_move, col_move)
        if simulated_game.win(current_player):
            return 1 if current_player == player else 0
        current_player = 1 if current_player == 2 else 2  # Switch player
    
    if simulated_game.win(current_player):
        return 1 if current_player == player else 0
    else:
        return 0 # Consider draw as half a win



----------------------------------------------------------------------------------------------------------------------------------

## Spider Line 4: Functional Game

In [ ]:

def main(mode):
    if mode == "pvp":
        board_size = display_board_size_menu()
        row_count, col_count = board_size, board_size

        board = Board(row_count, col_count)
        hovered_cell = None

        while not board.game_over:
            draw_board(board, hovered_cell)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

                elif event.type == pygame.MOUSEMOTION:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    col = (mouse_x - (WINDOW_WIDTH - board.col_count * CELL_SIZE) // 2) // CELL_SIZE
                    row = (mouse_y - (WINDOW_HEIGHT - board.row_count * CELL_SIZE) // 2) // CELL_SIZE
                    if 0 <= row < board.row_count and 0 <= col < board.col_count:
                        if [row, col] in board.actions():
                            hovered_cell = [row, col]
                        else:
                            hovered_cell = None
                    else:
                        hovered_cell = None

                elif event.type == pygame.MOUSEBUTTONDOWN:
                        mouse_x, mouse_y = pygame.mouse.get_pos()
                        col = (mouse_x - (WINDOW_WIDTH - board.col_count * CELL_SIZE) // 2) // CELL_SIZE
                        row = (mouse_y - (WINDOW_HEIGHT - board.row_count * CELL_SIZE) // 2) // CELL_SIZE

                        if [row, col] in board.actions():
                            board.put_piece(board.turn + 1, row, col)

                            if board.win(board.turn + 1):
                                print(f"Player {board.turn + 1} wins!")
                                board.game_over = True

                            if board.is_full():
                                print("The game is a draw!")
                                board.game_over = True
                                break

                            board.turn = 1 - board.turn
                            print("Array after click:\n", board.board)


        draw_board(board, None)
        pygame.time.wait(3000)
        pygame.quit()


    elif mode == "pvc":
        board_size = display_board_size_menu()
        row_count, col_count = board_size, board_size

        board = Board(row_count, col_count)
        hovered_cell = None

        difficulty = display_difficulty_menu("Choose Difficulty")

        while not board.game_over:
            draw_board(board, hovered_cell)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

                if board.turn == 0:
                    if event.type == pygame.MOUSEMOTION:
                        mouse_x, mouse_y = pygame.mouse.get_pos()
                        col = (mouse_x - (WINDOW_WIDTH - board.col_count * CELL_SIZE) // 2) // CELL_SIZE
                        row = (mouse_y - (WINDOW_HEIGHT - board.row_count * CELL_SIZE) // 2) // CELL_SIZE
                        if 0 <= row < board.row_count and 0 <= col < board.col_count:
                            if [row, col] in board.actions():
                                hovered_cell = [row, col]
                            else:
                                hovered_cell = None
                        else:
                            hovered_cell = None

                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        if board.turn == 0:
                            mouse_x, mouse_y = pygame.mouse.get_pos()
                            col = (mouse_x - (WINDOW_WIDTH - board.col_count * CELL_SIZE) // 2) // CELL_SIZE
                            row = (mouse_y - (WINDOW_HEIGHT - board.row_count * CELL_SIZE) // 2) // CELL_SIZE

                            if [row, col] in board.actions():
                                board.put_piece(board.turn + 1, row, col)

                                if board.win(board.turn + 1):
                                    print(f"Player {board.turn + 1} wins!")
                                    board.game_over = True

                                if board.is_full():
                                    print("The game is a draw!")
                                    board.game_over = True
                                    break

                                board.turn = 1 - board.turn
                                print("Array after click:\n", board.board)
                    
                # CPU's turn
                else:

                    # Define player_1 and player_2 for clarity
                    player_1 = 1
                    player_2 = 2
                    current_player = board.turn + 1  # This adjusts the player number correctly for the Minimax call

                    if difficulty == "easy_(mcts)":
                        c = 1.4
                        best_move = monte_carlo_tree_search(board, current_player,c, 10000, row_count, col_count)
                        row, col = best_move

                    elif difficulty == "hard_(minimax)":
                        _,row,col = minimax(board, 3, current_player, row_count, col_count, player_1,player_2,alpha=float('-inf'),beta=float('inf'))

                    elif difficulty == "hardcore_(negamax)":
                        _,row,col = negamax(board, 3, current_player, row_count, col_count, player_1,player_2,alpha=float('-inf'),beta=float('inf'))
                        

                    board.put_piece(board.turn + 1, row, col)

                    if board.win(board.turn + 1):
                        print(f"Player {board.turn + 1} wins!")
                        board.game_over = True

                    if board.is_full():
                        print("The game is a draw!")
                        board.game_over = True
                        break

                    board.turn = 1 - board.turn
                    print("Array after CPU's move:\n", board.board)

        draw_board(board, None)
        pygame.time.wait(3000)
        pygame.quit() 



    elif mode == "cvc":
        board_size = display_board_size_menu()
        row_count, col_count = board_size, board_size

        board = Board(row_count, col_count)
        hovered_cell = None

        difficulty1 = display_difficulty_menu("Choose Difficulty for the 1st algorithm")
        difficulty2 = display_difficulty_menu("Choose Difficulty for 2nd algorithm")


        while not board.game_over:

            draw_board(board, None)
            pygame.time.wait(2000)
            

            if board.turn == 0:
                # Define player_1 and player_2 for clarity
                player_1 = 1
                player_2 = 2
                current_player = board.turn + 1  # This adjusts the player number correctly for the Minimax call

                if difficulty1 == "easy_(mcts)":
                        c = 1.4
                        best_move = monte_carlo_tree_search(board, current_player,c, 10000, row_count, col_count)
                        row, col = best_move

                elif difficulty1 == "hard_(minimax)":
                    _,row,col = minimax(board, 3, current_player, row_count, col_count, player_1,player_2,alpha=float('-inf'),beta=float('inf'))

                elif difficulty1 == "hardcore_(negamax)":
                    _,row,col = negamax(board, 3, current_player, row_count, col_count, player_1,player_2,alpha=float('-inf'),beta=float('inf'))
                        

                board.put_piece(board.turn + 1, row, col)

                if board.win(board.turn + 1):
                    print(f"Player {board.turn + 1} wins!")
                    board.game_over = True

                if board.is_full():
                    print("The game is a draw!")
                    board.game_over = True
                    break

                board.turn = 1 - board.turn
                print("Array after CPU's move:\n", board.board)


            else:

                # Define player_1 and player_2 for clarity
                player_1 = 1
                player_2 = 2
                current_player = board.turn + 1  # This adjusts the player number correctly for the Minimax call

                if difficulty2 == "easy_(mcts)":
                        c = 1.4
                        best_move = monte_carlo_tree_search(board, current_player,c, 10000, row_count, col_count)
                        row, col = best_move

                elif difficulty2 == "hard_(minimax)":
                    _,row,col = minimax(board, 3, current_player, row_count, col_count, player_1,player_2,alpha=float('-inf'),beta=float('inf'))

                elif difficulty2 == "hardcore_(negamax)":
                    _,row,col = negamax(board, 3, current_player, row_count, col_count, player_1,player_2,alpha=float('-inf'),beta=float('inf'))
                    

                board.put_piece(board.turn + 1, row, col)

                if board.win(board.turn + 1):
                    print(f"Player {board.turn + 1} wins!")
                    board.game_over = True

                if board.is_full():
                    print("The game is a draw!")
                    board.game_over = True
                    break

                board.turn = 1 - board.turn
                print("Array after CPU's move:\n", board.board)

        draw_board(board, None)
        pygame.time.wait(3000)
        pygame.quit() 
    

if __name__ == "__main__":
    mode = display_menu()
    print(mode)
    main(mode)


pvp
Array after click:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Array after click:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Array after click:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Array after click:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 2. 0. 0.

: 

----------------------------------------------------------------------------------------------------------------------------------